In [1]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import pandas as pd
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys



In [2]:

driver = webdriver.Chrome('./chromedriver.exe')
driver.get('https://www.instagram.com/')

In [3]:

id_='seulyeon1122'
password_='jksy05312!'
ID=driver.find_element_by_css_selector("#loginForm > div > div:nth-child(1) > div > label > input")
ID.send_keys(id_)
PW=driver.find_element_by_css_selector("#loginForm > div > div:nth-child(2) > div > label > input")
PW.send_keys(password_)
login=driver.find_element_by_css_selector("#loginForm > div > div:nth-child(3)")
login.click()
time.sleep(3)
login2=driver.find_element_by_css_selector("#react-root > section > main > div > div > div > div > button") #로그인정보 저장 창
login2.click()
time.sleep(3)
login3=driver.find_element_by_css_selector("body > div.RnEpo.Yx5HN > div > div > div > div.mt3GC > button.aOOlW.HoLwm")#알림설정 창
login3.click()


In [4]:
time.sleep(2)
search=driver.find_element_by_css_selector("#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > input")
search.send_keys('bitamin_official')


In [5]:
time.sleep(2)
search.send_keys(Keys.ENTER)
search.send_keys(Keys.ENTER)
time.sleep(5)

In [10]:
def click_first(driver):
    first = driver.find_element_by_css_selector('#react-root > section > main > div > div._2z6nI > article > div:nth-child(1) > div > div:nth-child(1) > div:nth-child(1) > a > div.eLAPa > div._9AhH0')
    first.click()
    time.sleep(3)


In [47]:
def get_content(driver):

    # 1. 현재 게시글 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')


    # 2. 본문 내용 가져오기
    # 본문 내용이 없을 수 있으므로 예외 처리구문을 이용
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ''

    # 3. 작성 일시, 좋아요 수 , 위치 정보 가져오기
    # 해쉬 태그는 정규 표현식을 이용해 가져온다.
    try:
        tags = soup.select('div.C4VMK > span')[1].text
    except:
        tags = ''
    tags = re.findall(r'#[^\s#,\\]+', tags)  

    #  작성일자 정보 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]

    #  좋아요 수 가져오기
    # 예외처리구문.
    try:
        like = soup.select('div.Nm9Fw')[0].text[4:-1]   
    except:
        like = 0
    # 위치정보
    data = [content, date, like, tags]
    return data

In [16]:
def next_page(driver):
    next_page = driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.EfHg9 > div > div > a._65Bje.coreSpriteRightPaginationArrow')
    next_page.click()
    time.sleep(3)

In [48]:
click_first(driver)
result=[]
for i in range(6):
    data=get_content(driver)
    result.append(data)
    next_page(driver)

In [49]:
result_dataframe = pd.DataFrame(result)
result_dataframe.columns = ['본문','날짜','좋아요','해쉬태그']
result_dataframe

,본문,날짜,좋아요,해쉬태그
0,✨렛유인' X '비타민(BITAmin)✨<이공계 전문...,2021-02-22,14,"[#비타민, #bitamin, #빅데이터, #데이터분석, #머신러닝, #연합동아리,..."
1,🎉 2월 20일 세션 공지 🎉이번 주 세션을 공지합니다👋이번 주 6기는 '웹 크롤링...,2021-02-19,22,"[#비타민, #bitamin, #빅데이터, #연합동아리, #동아리, #대외활동, #..."
2,🍊비타민 2021-1학기 자율세션🍊오늘부터 비타민 2021년도 1학기 자율세션 신청...,2021-02-12,20,"[#비타민, #bitamin, #빅데이터, #데이터분석, #머신러닝, #연합동아리,..."
3,2월 6일 종강총회2월 6일에 비타민 5기와 6기의 종강총회가 있었습니다!사회적 거...,2021-02-10,55,"[#비타민, #bitamin, #빅데이터, #데이터분석, #머신러닝, #연합동아리,..."
4,"[면접 공지]안녕하세요, 비타민 운영진ᄋ...",2021-02-05,36,"[#비타민, #bitamin, #빅데이터, #데이터분석, #머신러닝, #연합동아리,..."
5,"🍊2월 6일 종강총회🍊이번 주는 5, 6기가 같이 모일 수 있는 마지막 날인데요😭바...",2021-02-04,18,"[#비타민, #bitamin, #빅데이터, #데이터분석, #머신러닝, #연합동아리,..."


In [51]:
import unicodedata
for i in range(6):
    result_dataframe['본문'][i]=unicodedata.normalize('NFC', result_dataframe['본문'][i]) #자모음 분리 방지

In [53]:
result_dataframe.to_csv("1주차 복습과제_6기_김동근.csv",encoding='utf-8-sig')